In [20]:
import pandas as pd
import numpy as np
import json
import random
import numpy as np
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
from keras.models import load_model
import sqlalchemy
import datetime
import spacy
import warnings
warnings.filterwarnings('ignore')

In [3]:
intents=json.loads(open('online.json').read())
nlp=spacy.load("en_core_web_sm")

In [4]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']
words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))

In [5]:
model=load_model('online.h5')

2023-02-07 22:34:33.003589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow=bag_of_words(sentence)
    # print(model.predict(np.array([bow])))
    res=model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list
def get_response(intents_list,intents_json):
    tag=intents_list[0]['intent']
    list_of_intents=intents_json['online']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result


In [7]:
mode=input("Enter mode: offline or online")
if((mode.lower())=="offline"):
    mode="offline"
else:
    mode="online"

In [8]:
flight_booking_no=1
flight_cancellation_no=0

In [9]:
# pid=flight_booking_no
# pName=""
# pPhone=""
# pEmail=""
# pSex=""
# pOrigin=""
# pDestiny=""
# pDate=""

In [10]:
# if(mode=="online"):
#     print("What would you like to do?")
#     print("1 : Book a Flight")
#     print("2 : Change a flight")
#     print("3 : Cancel a flight")
#     operation=""
#     while True:
#         message=input("Enter what you want to do:")
#         ints=predict_class(message)
#         res=get_response(ints,intents)
#         s=ints[0]['intent']
#         print(f'So you want to {s} the flight')
#         print("If i understood correctly then enter 'y' else enter 'n' to re-enter your demand")
#         check=input("Please Enter 'y' or 'n':")
#         if(check=="y"):
#             operation=s
#             break
#     print("I would like to ask u few things:")
#     if(operation=="book"):
#         print("FLIGHT BOOKING DETAILS.. HURRAY!!")
#         pID=flight_booking_no
#         flight_booking_no+=1
#         pName=input("Enter your name:")
#         pPhone=input("Enter your phone number:")
#         pEmail=input("Enter your email address:")
#         pSex=input("Enter your Sex")
#         pOrigin=input("Enter your Origin")
#         pDestiny=input("Enter your Destiny")
#         pDate=input(datetime.date)
#         # Database mein entry
#     elif(operation=="change"):
#         print("FLIGHT CHANGED DETAILS..It's Fine!!")
#         print("Would you like to change the date..?? If 'yes' then enter'y' otherwise enter 'n'")
#         check=input("Please Enter 'y' or 'n':")
#         if(check=='y'):
#             pDate=input(datetime.date)
#             print(f'So your new boarding date is {pDate}')
#         print("Would you like to change the destination..??")
#         check=input("Please Enter 'y' if u want to change or 'n':")
#         if(check=='y'):
#             pDestiny=input("Enter your new destination")
#             print(f'So your destination is {pDestiny}')
#         print("Details Updated")
#     else:
#         flight_cancellation_no+=1
#         print("FLIGHT CANCELED DETAILS..It's OK!!")
#         pidd=input("Please give me your passenger id:")
#         pnamee=input("Enter your name:")
#         pemaill=input("Enter your email address:")
#         pphonee=input("Enter your phone number:")
#         print("Thank You, we are cancelling your flight..")
#         print(f'Flight cancelled for {pnamee} id: {pidd}')
        

In [11]:
# text="I would like to go from mathura to new delhi on 2003-12-04 and email is abc12@gmail.com"
# doc=nlp(text)
# jagah=[]
# for ent in doc.ents :
#     print(ent.text,ent.label_)


In [12]:
# jagah

In [13]:
# ent.label_

In [14]:
places_code={
    'New Delhi':1,
    'Mathura':2,
    'Kanpur':3,
    'Varanasi':4,
    'Allahabad':5,
    'Jabalpur':6,
    'Agra':7
}

In [15]:
intent_id={
    'book':1,
    'change':2,
    'cancel':3
}

In [16]:
def process_chat_input(message):
    global flight_booking_no
    global flight_cancellation_no
    places=[]
    dates=[]
    operation=""
    pid=flight_booking_no
    pName=""
    pPhone=""
    pEmail=""
    pSex=""
    pOrigin=""
    pDestiny=""
    pBookingDate=""
    pDepartureDate="" 
    doc=nlp(message)
    for ent in doc.ents:
        if(ent.label_=="GPE"):
            places.append(ent.text)
        if(ent.label_=="DATE"):
            dates.append(ent.text)
    ints=predict_class(message)
    res=get_response(ints,intents)
    s=ints[0]['intent']
    print(f'So you want to {s} the flight having id {intent_id[s]}')
    if(s=="book"):
        flight_booking_no+=1
        print("FLIGHT BOOKING DETAILS.. HURRAY!!")
        pID=flight_booking_no
        flight_booking_no+=1
        pName=input("Enter your name:")
        pPhone=input("Enter your phone number:")
        pEmail=input("Enter your email address:")
        pSex=input("Enter your Sex")
        if(len(places)==0):
            pOrigin=input("Enter your Origin")
            pDestiny=input("Enter your Destiny")
        elif(len(places)==1):
            pOrigin=input("Enter your Origin")
            pDestiny=places[0]
        else:
            pOrigin=places[0]
            pDestiny=places[1]
        pBookingDate=datetime.date.today().strftime("%d-%m-%Y")
        if(len(dates)==0):
            pDepartureDate=input(datetime.date)
        else:
            pDepartureDate=dates[0]
        
        print(f'Your booking details are as follows:')
        print(f'ID {pid}')
        print(f'Name {pName}')    
        print(f'Phone {pPhone}')
        print(f'Email {pEmail}')
        print(f'Sex {pSex}')
        print(f'Origin {pOrigin} having code {places_code[pOrigin]}')
        print(f'Destiny {pDestiny} having code {places_code[pDestiny]}')
        print(f'BookingDate {pBookingDate}')
        print(f'DepartureDate {pDepartureDate}')
        
    elif(s=="change"):
        print(s)
        print("FLIGHT CHANGED DETAILS..It's Fine!!")
        print("Would you like to change the date..?? If 'yes' then enter'y' otherwise enter 'n'")
        check=input("Please Enter 'y' or 'n':")
        if(check=='y'):
            pDepartureDate=input(datetime.date)
            print(f'So your new boarding date is {pDepartureDate}')
        print("Would you like to change the origin..??")
        check=input("Please Enter 'y' if u want to change or 'n':")
        if(check=='y'):
            pOrigin=input("Enter your new destination")
            print(f'So your destination is {pOrigin}')
        print("Would you like to change the destination..??")
        check=input("Please Enter 'y' if u want to change or 'n':")
        if(check=='y'):
            pDestiny=input("Enter your new destination")
            print(f'So your destination is {pDestiny}')
        print("Details Updated") 
        
        print(f'Your booking details are as follows:')
        print(f'ID {pid}')
        print(f'Name {pName}')    
        print(f'Phone {pPhone}')
        print(f'Email {pEmail}')
        print(f'Sex {pSex}')
        print(f'Origin {pOrigin}')
        print(f'Destiny {pDestiny} having id {places_code[pDestiny]}')
        print(f'BookingDate {pBookingDate} having id {places_code[pBookingDate]}')
        print(f'DepartureDate {pDepartureDate}')
        
    elif(s=="cancel"):
        # print(s)
        flight_cancellation_no+=1
        print("FLIGHT CANCELLATION DETAILS..It's OK!!")
        pidd=input("Please give me your passenger id:")
        pnamee=input("Enter your name:")
        pemaill=input("Enter your email address:")
        pphonee=input("Enter your phone number:")
        print("Thank You, we are cancelling your flight..")
        print(f'Flight cancelled for {pnamee} id: {pidd}')
        

In [19]:
if mode.lower()=="online":
    def run_online_chatbot():
        while True:
            user_input=input("What can I help u with..??")
            if user_input.lower()=="exit":
                break
            else:
                process_chat_input(user_input)
                check=input("Would you like to book another ticket? (y/n): ")
                if(check.lower()=="y"):
                    pass
                else:
                    break
    run_online_chatbot()
else:
    def run_offline_chatbot():
        pass
        

1/1 [==============================] - 0s 21ms/step
So you want to book the flight having id 1
FLIGHT BOOKING DETAILS.. HURRAY!!
Your booking details are as follows:
ID 3
Name d
Phone d
Email d
Sex d
Origin d
Destiny d
BookingDate 07-02-2023
DepartureDate 12-12-2333


In [24]:
df.head()

,Intent,Name,PhoneNo,Email,Sex,Origin,Destiny,DepartureDate,pid
0,book ticket,parth,84331.0,scvd,dvbf,mathura,delhi,20-12-2003,NaN
1,change flight,NaN,NaN,NaN,NaN,delhi,lucknow,21-12-2003,1.0


In [33]:
IntentID=""
Name=""	
PhoneNo=""	
Email=""	
Sex=""
BookingDate=datetime.date.today().strftime("%d-%m-%Y")
DepartureDate=""
Origin=""
Destiny=""	
pid=0

In [51]:
def offline_chat(message):
    ints=predict_class(message)
    res=get_response(ints,intents)
    s=ints[0]['intent']
    print(f'So you want to {s} the flight having id {intent_id[s]}')
    return s
        

In [52]:
df.head()

,Intent,Name,PhoneNo,Email,Sex,Origin,Destiny,DepartureDate,pid
0,book ticket,parth,84331,scvd,dvbf,mathura,delhi,20-12-2003,NaN


In [57]:
df=pd.read_csv("offline.csv")
intent=df['Intent'][0]
# for intent in df['Intent']:
int=offline_chat(intent)
file = open("response.txt", "w")
if(int=="book"):
    print("Book a Flight")
    IntentID=intent_id[int]
    PhoneNo=df['PhoneNo'][0]
    Email=df['Email'][0]
    Sex=df['Sex'][0]
    Name=df['Name'][0]
    BookingDate=datetime.date.today().strftime("%d-%m-%Y")
    DepartureDate=df['DepartureDate']
    Origin=df['Origin'][0]
    Destiny=df['Destiny'][0]
    pid=flight_booking_no
    flight_booking_no+=1
    file.write(f'IntentID={IntentID}\n')
    file.write(f'Name={Name}\n')
    file.write(f'PID={pid}\n')
    file.write(f'PhoneNo={PhoneNo}\n')
    file.write(f'Email={Email}\n')
    file.write(f'Sex={Sex}\n')
    file.write(f'Origin={Origin}\n')
    file.write(f'Destiny={Destiny}\n')
    file.write(f'BookingDate={BookingDate}\n')
    file.write(f'DepartureDate={DepartureDate}\n')
elif(int=="change"):
    print("Change Flight Details")
    IntentID=intent_id[int]
    BookingDate=datetime.date.today().strftime("%d-%m-%Y")
    DepartureDate=df['DepartureDate']
    Origin=df['Origin'][0]
    Destiny=df['Destiny'][0]
    pid=flight_booking_no
    flight_booking_no+=1
    # file = open("response.txt", "w")
    file.write(f'IntentID={IntentID}\n')
    file.write(f'Name={Name}\n')
    file.write(f'PID={pid}\n')
    file.write(f'PhoneNo={PhoneNo}\n')
    file.write(f'Email={Email}\n')
    file.write(f'Sex={Sex}\n')
    file.write(f'Origin={Origin}\n')
    file.write(f'Destiny={Destiny}\n')
    file.write(f'BookingDate={BookingDate}\n')
    file.write(f'DepartureDate={DepartureDate}\n')
else:
    pid=df['pid']
    print('Flight cancelled')
        

1/1 [==============================] - 0s 35ms/step
So you want to book the flight having id 1
Book a Flight


In [49]:
df['Email']

0    scvd
Name: Email, dtype: object